In [1]:
import pandas as pd
import numpy as np
import os.path
from datetime import datetime
base_path = '../dataset'

original_data = os.path.join(base_path, 'original_data')
processed_data = os.path.join(base_path, 'processed_data')

In [2]:
candidate_items=pd.read_csv(os.path.join(original_data, "candidate_items.csv"),usecols=["item_id"]).drop_duplicates().sort_values(by="item_id").to_numpy()
candidate_items[:5]

array([[ 4],
       [ 8],
       [ 9],
       [19],
       [20]], dtype=int64)

In [3]:
train_sessions_items=pd.read_csv(os.path.join(original_data,"train_sessions.csv"),usecols=["item_id"]).drop_duplicates().sort_values(by="item_id").to_numpy()
train_sessions_items[:5]

array([[2],
       [3],
       [4],
       [8],
       [9]], dtype=int64)

In [4]:
train_purchases_items=pd.read_csv(os.path.join(original_data,"train_purchases.csv"),usecols=["item_id"]).drop_duplicates().sort_values(by="item_id").to_numpy()
train_purchases_items[:5]

array([[3],
       [4],
       [7],
       [8],
       [9]], dtype=int64)

In [5]:
test_sessions_items=pd.read_csv(os.path.join(original_data,"test_final_sessions.csv"),usecols=["item_id"]).drop_duplicates().sort_values(by="item_id").to_numpy()
test_sessions_items[:5]

array([[ 3],
       [ 4],
       [ 8],
       [ 9],
       [19]], dtype=int64)

In [6]:
test_sessions_leaderboard_items=pd.read_csv(os.path.join(original_data,"test_leaderboard_sessions.csv"),usecols=["item_id"]).drop_duplicates().sort_values(by="item_id").to_numpy()
test_sessions_leaderboard_items[:5]

array([[ 4],
       [ 8],
       [ 9],
       [19],
       [20]], dtype=int64)

In [7]:
total_test_items = np.concatenate((test_sessions_leaderboard_items, test_sessions_items))

In [8]:
InTestButNotInTrain=np.setdiff1d(total_test_items,train_sessions_items)
InTestButNotInTrain

array([  313,   366,   575,  1152,  1364,  1883,  2497,  2523,  2677,
        2694,  3185,  3529,  3754,  3835,  4042,  4514,  5214,  5394,
        6171,  6853,  6873,  6916,  7204,  7780,  8758,  8771,  9384,
        9418,  9589, 10463, 10671, 11125, 11933, 12667, 13376, 13618,
       13943, 13972, 14395, 14622, 14723, 14967, 15601, 15629, 16206,
       17046, 17206, 18482, 18690, 18837, 19637, 21241, 21444, 21904,
       21927, 21998, 22030, 22316, 22703, 22746, 24303, 25035, 25277,
       25521, 26232, 26742, 27377, 27728, 27826], dtype=int64)

In [9]:
print(len(InTestButNotInTrain)," items are in test sessions but not in train")

69  items are in test sessions but not in train


In [10]:
np.intersect1d(InTestButNotInTrain,train_purchases_items)
#None of those 67 items is seen in the train_purchases, so they are unknown items, should be treated in a special way

array([], dtype=int64)

In [11]:
CandidateNeverBoughtInTrainingSet=np.setdiff1d(candidate_items,train_purchases_items)
CandidateNeverBoughtInTrainingSet

array([  221,   313,   344,   366,   575,   950,  1152,  1222,  1364,
        1883,  1935,  2411,  2489,  2497,  2523,  2621,  2677,  2694,
        3185,  3220,  3529,  3754,  3835,  4042,  4514,  6171,  6425,
        6512,  6853,  6873,  6916,  7204,  7321,  7408,  7710,  7780,
        8053,  8758,  8814,  8839,  8928,  9384,  9418,  9589,  9974,
       10463, 10491, 10496, 10671, 10694, 11125, 11862, 11933, 12438,
       12641, 12667, 13376, 13550, 13618, 13788, 13943, 13972, 14167,
       14395, 14535, 14723, 15629, 15719, 16045, 16092, 16206, 16800,
       17046, 17057, 17206, 17371, 17534, 17576, 17950, 18837, 19808,
       20237, 21056, 21241, 21444, 21599, 21904, 21998, 22096, 22143,
       22316, 22583, 22703, 22746, 22838, 23640, 23981, 24239, 24303,
       25035, 25179, 25277, 25298, 25521, 26201, 26232, 26742, 27042,
       27377, 27427, 27728, 27826, 27847], dtype=int64)

In [12]:
len(CandidateNeverBoughtInTrainingSet)

113

In [13]:
SeenInTrainSessions=np.intersect1d(CandidateNeverBoughtInTrainingSet,train_sessions_items)
SeenInTrainSessions

array([  221,   344,   950,  1222,  1935,  2411,  2489,  2621,  3220,
        6425,  6512,  7321,  7408,  7710,  8053,  8839,  8928,  9974,
       10491, 10496, 10694, 11862, 12438, 12641, 13550, 13788, 14167,
       14535, 15719, 16045, 16092, 16800, 17057, 17371, 17534, 17950,
       19808, 21056, 21599, 22096, 22143, 22583, 22838, 23640, 23981,
       24239, 25179, 25298, 27042, 27427, 27847], dtype=int64)

In [14]:
len(SeenInTrainSessions)

51

In [15]:
data=np.concatenate((candidate_items,train_purchases_items,train_sessions_items,test_sessions_items,test_sessions_leaderboard_items))


In [16]:
df=pd.DataFrame(columns=["item_id"],data=data,index=[i for i,_ in enumerate(data)]).drop_duplicates().reset_index()

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23691 entries, 0 to 23690
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   index    23691 non-null  int64
 1   item_id  23691 non-null  int64
dtypes: int64(2)
memory usage: 370.3 KB


In [18]:
df["code"]=df.index
df["code"]+=1

In [19]:
del df["index"]

In [20]:
df.to_csv(os.path.join(processed_data,"map.csv"),index=False)

In [21]:
df[:19021].to_csv(os.path.join(processed_data,"map_purchases.csv"),index=False) # Mapping of only bought items

In [22]:
train_sessions=pd.read_csv(os.path.join(original_data,"train_sessions.csv"),parse_dates=['date'], 
                         infer_datetime_format=True,header=0)
                         
train_sessions_ids=train_sessions.sort_values("date").drop_duplicates("session_id",keep='first')["session_id"]
train_sessions_ids=train_sessions_ids.reset_index()
train_sessions_ids["index"]=train_sessions_ids.index
train_sessions_ids.to_csv(os.path.join(processed_data,"sessions_map.csv"),index=False)

In [23]:
test_final_sessions=pd.read_csv(os.path.join(original_data,"test_final_sessions.csv"))
test_leaderboard_sessions=pd.read_csv(os.path.join(original_data,"test_leaderboard_sessions.csv"))
test_sessions=pd.concat([test_final_sessions,test_leaderboard_sessions],axis=0)

test_sessions_ids=test_sessions.sort_values("date").drop_duplicates("session_id",keep='first')["session_id"]
test_sessions_ids=test_sessions_ids.reset_index()
test_sessions_ids["index"]=test_sessions_ids.index
test_sessions_ids.to_csv(os.path.join(processed_data,"test_sessions_map.csv"),index=False)

In [24]:
candidate_items=pd.read_csv(os.path.join(original_data,"candidate_items.csv"))
candidate_items=candidate_items.merge(df,on="item_id",how="left")
candidate_items["item_id"]=candidate_items["code"]
del candidate_items["code"]
candidate_items.to_csv(os.path.join(processed_data,"candidate_items_mapped.csv"),index=False)

In [25]:
test_final_sessions=pd.read_csv(os.path.join(original_data,"test_final_sessions.csv"))
test_final_sessions=test_final_sessions.merge(df,on="item_id",how="left")
test_final_sessions["item_id"]=test_final_sessions["code"]
del test_final_sessions["code"]
test_final_sessions=test_final_sessions.merge(test_sessions_ids,on="session_id",how="left")
test_final_sessions["session_id"]=test_final_sessions["index"]
del test_final_sessions["index"]
test_final_sessions.to_csv(os.path.join(processed_data,"test_final_sessions_full_mapped.csv"),index=False)

In [ ]:
test_final_sessions=pd.read_csv(os.path.join(original_data,"test_final_sessions.csv"))
test_final_sessions=test_final_sessions.merge(df,on="item_id",how="left")
test_final_sessions["item_id"]=test_final_sessions["code"]
del test_final_sessions["code"]
test_final_sessions.to_csv(os.path.join(processed_data,"test_final_sessions_mapped.csv"),index=False)

In [26]:
test_leaderboard_sessions=pd.read_csv(os.path.join(original_data,"test_leaderboard_sessions.csv"))
test_leaderboard_sessions=test_leaderboard_sessions.merge(df,on="item_id",how="left")
test_leaderboard_sessions["item_id"]=test_leaderboard_sessions["code"]
del test_leaderboard_sessions["code"]
test_leaderboard_sessions=test_leaderboard_sessions.merge(test_sessions_ids,on="session_id",how="left")
test_leaderboard_sessions["session_id"]=test_leaderboard_sessions["index"]
del test_leaderboard_sessions["index"]
test_leaderboard_sessions.to_csv(os.path.join(processed_data,"test_leaderboard_sessions_full_mapped.csv"),index=False)

In [ ]:
test_leaderboard_sessions=pd.read_csv(os.path.join(original_data,"test_leaderboard_sessions.csv"))
test_leaderboard_sessions=test_leaderboard_sessions.merge(df,on="item_id",how="left")
test_leaderboard_sessions["item_id"]=test_leaderboard_sessions["code"]
del test_leaderboard_sessions["code"]
test_leaderboard_sessions.to_csv(os.path.join(processed_data,"test_leaderboard_sessions_mapped.csv"),index=False)

In [27]:
train_purchases=pd.read_csv(os.path.join(original_data,"train_purchases.csv"))
train_purchases=train_purchases.merge(df,on="item_id",how="left")
train_purchases["item_id"]=train_purchases["code"]
del train_purchases["code"]
train_purchases=train_purchases.merge(train_sessions_ids,on="session_id",how="left")
train_purchases["session_id"]=train_purchases["index"]
del train_purchases["index"]
train_purchases.to_csv(os.path.join(processed_data,"train_purchases_mapped.csv"),index=False)

In [28]:
train_sessions=pd.read_csv(os.path.join(original_data,"train_sessions.csv"))
train_sessions=train_sessions.merge(df,on="item_id",how="left")
train_sessions["item_id"]=train_sessions["code"]
del train_sessions["code"]
train_sessions=train_sessions.merge(train_sessions_ids,on="session_id",how="left")
train_sessions["session_id"]=train_sessions["index"]
del train_sessions["index"]
train_sessions.to_csv(os.path.join(processed_data,"train_sessions_mapped.csv"),index=False)

In [30]:
item_features=pd.read_csv(os.path.join(original_data,"item_features.csv"))
item_features=item_features.merge(df,on="item_id",how="left")
item_features["item_id"]=item_features["code"]
del item_features["code"]
item_features.to_csv(os.path.join(processed_data,"item_features_mapped.csv"),index=False)

In [31]:
train_sessions=pd.read_csv(os.path.join(original_data,"train_sessions.csv"),parse_dates=['date'], 
                         infer_datetime_format=True,header=0)


last_month_start = datetime.strptime("01/05/21 00:00:00", '%d/%m/%y %H:%M:%S')

train_sessions_train_split=train_sessions[train_sessions["date"]<last_month_start]


train_sessions_train_split=train_sessions_train_split.merge(df,on="item_id",how="left")
train_sessions_train_split["item_id"]=train_sessions_train_split["code"]
del train_sessions_train_split["code"]
train_sessions_train_split=train_sessions_train_split.merge(train_sessions_ids,on="session_id",how="left")
train_sessions_train_split["session_id"]=train_sessions_train_split["index"]
del train_sessions_train_split["index"]
train_sessions_train_split.to_csv(os.path.join(processed_data,"train_sessions_train_split_mapped.csv"),index=False)

train_sessions_valid_split=train_sessions[train_sessions["date"]>=last_month_start]


train_sessions_valid_split=train_sessions_valid_split.merge(df,on="item_id",how="left")
train_sessions_valid_split["item_id"]=train_sessions_valid_split["code"]
del train_sessions_valid_split["code"]
train_sessions_valid_split=train_sessions_valid_split.merge(train_sessions_ids,on="session_id",how="left")
train_sessions_valid_split["session_id"]=train_sessions_valid_split["index"]
del train_sessions_valid_split["index"]
train_sessions_valid_split.to_csv(os.path.join(processed_data,"train_sessions_valid_split_mapped.csv"),index=False)

In [32]:
train_purchases=pd.read_csv(os.path.join(original_data,"train_purchases.csv"),parse_dates=['date'], 
                         infer_datetime_format=True,header=0)


last_month_start = datetime.strptime("01/05/21 00:00:00", '%d/%m/%y %H:%M:%S')

train_purchases_train_split=train_purchases[train_purchases["date"]<last_month_start]


train_purchases_train_split=train_purchases_train_split.merge(df,on="item_id",how="left")
train_purchases_train_split["item_id"]=train_purchases_train_split["code"]
del train_purchases_train_split["code"]
train_purchases_train_split=train_purchases_train_split.merge(train_sessions_ids,on="session_id",how="left")
train_purchases_train_split["session_id"]=train_purchases_train_split["index"]
del train_purchases_train_split["index"]
train_purchases_train_split.to_csv(os.path.join(processed_data,"train_purchases_train_split_mapped.csv"),index=False)

train_purchases_valid_split=train_purchases[train_purchases["date"]>=last_month_start]


train_purchases_valid_split=train_purchases_valid_split.merge(df,on="item_id",how="left")
train_purchases_valid_split["item_id"]=train_purchases_valid_split["code"]
del train_purchases_valid_split["code"]
train_purchases_valid_split=train_purchases_valid_split.merge(train_sessions_ids,on="session_id",how="left")
train_purchases_valid_split["session_id"]=train_purchases_valid_split["index"]
del train_purchases_valid_split["index"]
train_purchases_valid_split.to_csv(os.path.join(processed_data,"train_purchases_valid_split_mapped.csv"),index=False)